In [ ]:
function parallel(nproc, seed)
    srand(seed)
    addprocs(nproc)
    initialvalues = randn(273, nproc)
    
    @everywhere include(“preparation.jl”)
    @everywhere include(“bayes.jl”)
    @everywhere include(“loglike.jl”)
    
    estparam = Array(Float64, 273, nproc)
    likelihood = Array(Float64, 1, nproc)
    result = Array(Any, nproc, 1)
    
    for i in 1:nproc
        result[i, 1] = remotecall(optimize, i, loglike, initialvalues[:, i], BFGS())
    end
    
    for j in 1:nproc
        z = fetch(result[j, 1])
        estparam[:, j] = Optim.minimizer(z)
        likelihood[1, j] = Optim.minimum(z)
    end
    
    writetable(*("estparam", "_$seed", ".txt"), estparam)
    writetable(*("likelihood", "_$seed", ".txt"), likelihood)

end

In [77]:
for i in 1:3
    println(*("$i", "a"))
end

1a
2a
3a


In [62]:
using DataFrames

In [78]:
q = DataFrame(randn(2,3))
writetable("kei.txt", q)

In [79]:
a = readtable("kei.txt")

,x1,x2,x3
1,0.40502413565838247,1.2816281942276413,-0.3967146272808418
2,-0.9368406702520177,0.7271425422661754,1.05541348144974


In [80]:
Array(a[:, 1])

2-element Array{Float64,1}:
  0.405024
 -0.936841

In [50]:
a = Array(Any, 3,1)
for i in 1:3
    a[i, 1] = remotecall(f, i, i)
end

In [53]:
fetch(a[3,1])

6

In [47]:
*("z", "1") = 

LoadError: LoadError: syntax: ""z"" is not a valid function argument name
while loading In[47], in expression starting on line 1

In [27]:

function f(x)
    return (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
end

f (generic function with 1 method)

In [30]:
using Optim
result = optimize(f, [0.0, 0.0])

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.9999634355313174,0.9999315506115275]
 * Minimum: 3.735744e-09
 * Iterations: 60
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 90


In [40]:
likelihood = Array(Float64, 1, 3)[1, 2]

2.2616560306e-314

In [35]:
a = Array(Float64, 2, 2)

2×2 Array{Float64,2}:
 2.25221e-314  2.25123e-314
 2.25227e-314  2.25112e-314

In [36]:
a[:, 1] = Optim.minimizer(result)

2-element Array{Float64,1}:
 0.999963
 0.999932

In [37]:
a

2×2 Array{Float64,2}:
 0.999963  2.25123e-314
 0.999932  2.25112e-314

In [34]:
Optim.minimum(result)

3.7357441674966805e-9

In [22]:
a = randn((2,3))

2×3 Array{Float64,2}:
 -0.339366  -0.888936  0.592403
 -0.843878   0.327215  0.368002

In [26]:
Array((2,3))

LoadError: LoadError: MethodError: Cannot `convert` an object of type Tuple{Int64,Int64} to an object of type Array{T,N}
This may have arisen from a call to the constructor Array{T,N}(...),
since type constructors fall back to convert methods.
while loading In[26], in expression starting on line 1

In [24]:
for i in 1:3
    println(i)
end

1
2
3


In [23]:
a[:, 2]

2-element Array{Float64,1}:
 -0.888936
  0.327215

In [41]:
@everywhere function f(x)
    return 2*x
end

In [1]:
# ワーカープロセスの追加
# 要するに1はマスタープロセスか
addprocs(2)


2-element Array{Int64,1}:
 2
 3

In [43]:
f1 = remotecall(2, f, 3)

Future(2,1,28,Nullable{Any}())

In [44]:
fetch(f1)

6

In [8]:
srand(123)

MersenneTwister(UInt32[0x0000007b],Base.dSFMT.DSFMT_state(Int32[1464307935,1073116007,222134151,1073120226,-290652630,1072956456,-580276323,1073476387,1332671753,1073438661  …  138346874,1073030449,1049893279,1073166535,-1999907543,1597138926,-775229811,32947490,382,0]),[1.13343,1.5014,1.78682,1.68388,1.10635,1.48042,1.07618,1.52502,1.3404,1.91849  …  1.39174,1.71989,1.95426,1.84418,1.29131,1.2316,1.70209,1.70052,1.16077,1.75096],382)

In [9]:
r1 = remotecall(rand, 1, 2, 2)

Future(1,1,9,Nullable{Any}())

In [10]:
r2 = remotecall(rand, 2, 2, 2)

Future(2,1,10,Nullable{Any}())

In [11]:
r3 = remotecall(rand, 3, 2, 2)

Future(3,1,11,Nullable{Any}())

In [12]:
# ワーカープロセスの数を超えちゃってるから出ない。
r4 = remotecall(rand, 4, 2, 2)

LoadError: LoadError: no process with id 4 exists
while loading In[12], in expression starting on line 1

In [13]:
fetch(r1)

2×2 Array{Float64,2}:
 0.768448  0.673959
 0.940515  0.395453

In [14]:
fetch(r2)

2×2 Array{Float64,2}:
 0.152437  0.601667 
 0.4353    0.0279176

In [15]:
fetch(r3)

2×2 Array{Float64,2}:
 0.972651  0.797308
 0.929411  0.871865

In [18]:
z = @spawnat 2 f(3)

Future(2,1,17,Nullable{Any}())

In [19]:
fetch(z)

6

In [20]:
z2 = @spawnat 3 f(3)
fetch(z2)

6